In [1]:
import keras
import keras.preprocessing
import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'keras'

In [2]:
# install keras
% pip install keras

UsageError: Line magic function `%` not found.


# Natural Language Processing with Python


In [57]:
#Download all the works of shakespear
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

## Next we encode every character as an integer using the Keras Tokenizer class.
This will find all the characters used in the text and map them to a different character ID from 1 to [number of unique characters]. Mind that it does not start at zero. 

Char_level = true means we're encoding characters instead of words.
It also converts text to lower case by default (set with lower=False or lower=True)

In [58]:
## Next we encode every character as an integer using the Keras Tokenizer class.
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) # char_level=True means that we tokenize characters rather than words or word subparts
tokenizer.fit_on_texts(shakespeare_text)

Here's an example of the tokenizer in action:

In [59]:
tokenizer.texts_to_sequences(["First"]) 


[[20, 6, 9, 8, 3]]

In [60]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]]) # The Tokenizer converts each character to a different integer, starting from 1

['f i r s t']

In [61]:
tokenizer.sequences_to_texts([[20, 6, 8, 3, 8]])

['f i s t s']

In [62]:
max_id=len(tokenizer.word_index)
print(max_id)

39


In [63]:
#dataset size
dataset_size = tokenizer.document_count
print(dataset_size)

1115394


So we see the complete works of Shakespeare is 1,115,394 characters. 

Next step is to encode them all to their unique IDs. We subtract 1 so we get 0 to 38 instead of 1 to 39.

In [64]:
[encoded]  = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [65]:
np.size(encoded)

1115394

Before continuing, we need to split into a training, validation, and test set as per usual, but...*how* in this case? 

When dealing with time series you would generally split accross time. With financial data maybe you split accross different companies over time....

It's not exactly a trivial decision to make. 

For now let's take the first 90% of the text for the training set and keep the rest for the validation and test set. 

In [66]:
train_size= dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

The data now consists of a single sequence of over a million characters. It'd be folly to directly train the RNN directly on it: the RNN would be equivilant to a deep layer of over a million layhers, and we would have a single (and VERY LONG) instance to train it. 

Instead, we will use the dataset's **window()** method to convert this long sequence of characters into many smaller windows of text. 

Every instance in the dataset will be a faily short substring of the whole text as a result, and the RNN wiwll be unrolled only over the length of these substrings. 

This is known as *truncated backpropagation*.

In [67]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset= dataset.repeat().window(window_length, shift=1, drop_remainder=True)

The window() method creates a *nested dataset* of windows, each of which is also represented as a dataset (think of it as a list of lists).

By default the window() method creates nonoverlapping windows, but to get the largest possible training set we use *shift=1* so that the first window contains characters 0-100, the second contains 1-101, and so on. This allows us to create character batches without needing to do any padding. 

We also set *drop_remainder=True* or else the last 100 windows will 100, then 99, then so on down to 1 character.

Mind you, we can't directly use a nested dataset directly for training, as our model will expect tensors as input. Therefore, we'll call upon the *flat_map()* method to convert these windows into a flat dataset.


In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

Now the dataset contains consecutive windows of 101 characters each. Since Gradient Descent works best when the training set instances are independent and identically distributed, we'll shuffle the windows.

In [70]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
#dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

#let's not forget to one-hot encode our data
# dataset = dataset.map(lambda batch: (tf.one_hot(batch[0], depth=max_id), batch[1]))



TypeError: in user code:


    TypeError: <lambda>() takes 1 positional argument but 2 were given


In [71]:
dataset = dataset.prefetch(1)

## Building and Training the Char-RNN Model

In [72]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                        dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                        dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)


Epoch 1/10
31370/31370 [==============================] - 10193s 324ms/step - loss: 3.0646
Epoch 2/10
31370/31370 [==============================] - 11761s 375ms/step - loss: 3.0640
Epoch 3/10
25394/31370 [=======================>......] - ETA: 37:26 - loss: 3.0638

In [ ]:
#save the model
model.save("my_shakespeare_model.h5")

In [1]:
def f(a, b):
    return a+b

f = lambda x:f(x, 0)